# BEVPlace++ — Minimal Demo

This notebook briefly demonstrates:
- Torch-based BEV density image generation
- REM + NetVLAD forward pass to produce global descriptors
- Simple retrieval with PCA + FAISS index

Note: This is a shape/flow demo; for speed and offline use, REM is instantiated without pretrained weights.


In [1]:
import torch
import numpy as np

from bevplace import (
    bev_density_image_torch,
    REIN,
)
from bevplace.core.types import BEVParams
from bevplace.retrieval import BEVIndex

# Select device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE


'cuda'

In [2]:
# 1) Generate a random point cloud and build BEV
N = 200000
pts = (torch.randn(N, 3, device=DEVICE) * 30.0)
params = BEVParams(D=40.0, g=0.4)
bev = bev_density_image_torch(pts, params)
bev.shape, bev.min().item(), bev.max().item(), bev.device


(torch.Size([1, 200, 200]), 0.0, 1.0, device(type='cuda', index=0))

In [3]:
# 2) REIN forward: local/global descriptors
model = REIN().to(DEVICE).eval()
with torch.no_grad():
    out1, out2, desc = model(bev.unsqueeze(0))  # [1,1,H,W] -> batch 1
out1.shape, out2.shape, desc.shape


(torch.Size([1, 128, 50, 50]),
 torch.Size([1, 128, 200, 200]),
 torch.Size([1, 8192]))

In [7]:
# 3) Simple retrieval: build small DB and query
rng = np.random.default_rng(0)
# Make 10 random descriptors around our query desc with small noise
base = desc.detach().cpu().numpy()
DB = np.vstack([
    base + rng.normal(scale=0.1, size=base.shape)
    for _ in range(10)
])  # 10 items, each with independent noise

index = BEVIndex(pca_dim=16)
index.fit_pca(DB)
index.add(DB, poses=None)

hits = index.search(base, k=3)
hits[0]  # show top-3


[RetrievalHit(id=1, distance=72.49774932861328),
 RetrievalHit(id=7, distance=72.54429626464844),
 RetrievalHit(id=9, distance=72.81539154052734)]